In [9]:
import pandas as pd
import numpy as np
import mysql.connector
import os
from mysql.connector import Error
from datetime import datetime, date, timedelta
import re
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [10]:
#Definimos la ruta en donde queremos guardar los archivos
ruta_guardar = '/home/estadistico/Documents/Erick/Reportes diarios'
ruta_asig_baz = '/home/estadistico/Documents/Erick/Banco Azteca/Asignacion csv'
ruta_pagos = '/home/estadistico/Documents/Erick'
mes = 20201201
mes2 = 'Diciembre2020'
mes3 = pd.to_datetime('01/12/2020',dayfirst=True)
prim_sem_mes = 49

In [11]:
#Agregamos las variables a ocupar
servidor = '192.168.15.12'
puerto = int('3306')
usuario = 'estadisticas'
contrasena = 'estadisticas8474'
base = 'procesos_externos'

In [12]:
#Lectura pagos Bradesco c0
pagos_brad_c0 = pd.read_excel(''+ruta_pagos+'/Bradesco/Facturacion_c0/Facturacion c0.xlsx',converters={'TARJETA':str})
pagos_brad_c0 = pagos_brad_c0.drop_duplicates()
#pagos_brad2 = pd.pivot_table(pagos_brad,index=['NUMERO_TARJETA','FECHA_PAGO','TIPO_ASIGANCION'],values=['MONTO'],aggfunc=np.sum)
#pagos_brad2 = pd.DataFrame(pagos_brad2.to_records())
#pagos_brad2 = pagos_brad2.loc[pagos_brad2['FECHA_PAGO']>=mes3]

In [13]:
#Lectura pagos Bradesco extrajudicial
pagos_brad_extra = pd.read_excel(''+ruta_pagos+'/Bradesco/Facturacion_extra/Facturacion extra.xlsx',converters={'TARJETA':str})
pagos_brad_extra = pagos_brad_extra.drop_duplicates()
#pagos_brad2 = pd.pivot_table(pagos_brad,index=['NUMERO_TARJETA','FECHA_PAGO','TIPO_ASIGANCION'],values=['MONTO'],aggfunc=np.sum)
#pagos_brad2 = pd.DataFrame(pagos_brad2.to_records())
#pagos_brad2 = pagos_brad2.loc[pagos_brad2['FECHA_PAGO']>=mes3]

In [14]:
#Lectura pagos Liverpool
pagos_liv = pd.read_excel(''+ruta_pagos+'/Liverpool/Pagos/Facturacion Liverpool.xlsx',converters={'Cuenta':str})

In [15]:
#Asignamos valores a los parametros \n"
today = date.today().strftime('%Y%m%d')
now = datetime.now().strftime('%d-%m-%Y %H:%M')
hoy = np.datetime64('today')
ayer = np.datetime64(date.today() - timedelta(1))
today2 = date.today().strftime('%d-%m-%Y')

In [16]:
#Hacemos la conexion con el servidor\n",
try: 
    conn = mysql.connector.connect(user=usuario,
                               password=contrasena,
                               host=servidor,
                               port=3306,
                               database=base)
    conn.set_charset_collation('latin1')
except mysql.Error as e:
    print("Failed to execute stored procedure: {}".format(error))

cursor = conn.cursor()

In [17]:
#Funcion para hacer la consulta sql, ya sea de gestion o de asignacion en donde requerimos tres parametros, la tabla de donde vamos a sacar la consulta, en caso de asignacin requerimos los otros dos parametros fecha inicial y fecha final
def consulta_sql(table,from_date=None,to_date=None,campana=None):
    if campana is None:
        if from_date is None:
            sql_mes = cursor.callproc(table,)
            for result in cursor.stored_results():
                sql_mes = pd.DataFrame(result.fetchall())
        else:
            sql_mes = cursor.callproc(table, [from_date,to_date])
            for result in cursor.stored_results():
                sql_mes = pd.DataFrame(result.fetchall())
    else:
        sql_mes = cursor.callproc(table, [from_date,to_date,campana])
        for result in cursor.stored_results():
                sql_mes = pd.DataFrame(result.fetchall())
    sql_mes = sql_mes.drop_duplicates()
    return sql_mes

In [18]:
#Funcion para transformar varias columnas a numero
def transform_numeric(df,cols):
    return df[cols].apply(pd.to_numeric, errors='coerce')

#Funcion que convierte a dataframe y rellena espacios con 0
def to_dataframe(df):
    df = pd.DataFrame(df.to_records())
    df.fillna(0,inplace=True)
    return df

#Funcion que quita el % del string
def percent_tonumeric(df,cols):
    for c in cols:
        df[c] = [x[:-2] for x in df[c]]
    return transform_numeric(df,cols)

# Bradesco

In [19]:
#Inicio y fin de meses que nos interesa conocer
inicio_mes = [20200101,20200201,20200301,20200401,20200501,20200601,20200701,20200801,20200901,20201001,20201101]
fin_mes = [20200131,20200228,20200331,20200430,20200531,20200630,20200731,20200831,20200930,20201031,20201130]

In [20]:
#Lectura de cada uno de los meses
gestion_brad = pd.DataFrame()
for i,j in zip(inicio_mes,fin_mes):
    gestion_brad_mes = consulta_sql('bradescard_rpt_gestiones_detallado',i,j)
    gestion_brad = pd.concat([gestion_brad,gestion_brad_mes],ignore_index=True)
    print(i)

20200101
20200201
20200301
20200401
20200501
20200601
20200701
20200801
20200901
20201001
20201101


In [21]:
#Asignacion de nombre de columnas
gestion_brad.columns = ['folio_gestion','firma_id','unegocio_id','credito','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion','resultado','accion_resultado','fecha_promesa','monto_promesa','comentarios']
promesas_brad = gestion_brad.loc[gestion_brad['dictamen']=='PROMESA']

In [22]:
#Lectura de cada uno de los meses
gestion_liv = pd.DataFrame()
for i,j in zip(inicio_mes,fin_mes):
    gestion_liv_mes = consulta_sql('liverpool_rpt_gestiones_detallado',i,j)
    gestion_liv = pd.concat([gestion_liv,gestion_liv_mes],ignore_index=True)
    print(i)

20200101
20200201
20200301
20200401
20200501
20200601
20200701
20200801
20200901
20201001
20201101


In [23]:
gestion_liv.columns = ['folio_gestion','firma_id','unegocio_id','credito','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion','resultado','fecha_promesa','monto_promesa','comentarios']
promesas_liv = gestion_liv.loc[gestion_liv['dictamen']=='PROMESA']

In [24]:
#Ajustamos parametros para la gestion
promesas_brad_liv = pd.concat([promesas_brad,promesas_liv],ignore_index=True)
promesas_brad_liv['monto_promesa'] = pd.to_numeric(promesas_brad_liv['monto_promesa'])
promesas_brad_liv = promesas_brad_liv.drop_duplicates(subset=['credito','usuario','fecha_promesa','monto_promesa'])
promesas_brad_liv['anio'] = [x[:4] for x in promesas_brad_liv['fecha_promesa']]
promesas_brad_liv = promesas_brad_liv.loc[(promesas_brad_liv['anio']=='2020')]
promesas_brad_liv['fecha_promesa'] = pd.to_datetime(promesas_brad_liv['fecha_promesa'],dayfirst=False)
promesas_brad_liv['mes'] = pd.DatetimeIndex(promesas_brad_liv['fecha_promesa']).month
promesas_brad_liv['credito_unico'] = promesas_brad_liv['credito'].map(str)+'-'+promesas_brad_liv['mes'].map(str)+'-'+promesas_brad_liv['anio'].map(str)
promesas_brad_liv.head(1)

,folio_gestion,firma_id,unegocio_id,credito,nombre_credito,telefono,tipo_telefono,fecha_gestion,usuario,nombre_usuario,dictamen,accion,resultado,accion_resultado,fecha_promesa,monto_promesa,comentarios,anio,mes,credito_unico
0,6604600,1,4,5063290226839017,ORONA PONCE ANGEL ALFREDO,6674234755,celular,2020-01-31 20:43:22,YHERNANDEZBA,ALEJANDRA YESENIA HERNANDEZ BITE,PROMESA,LLAMADA A TITULAR,PROMESA DE PAGO LIQUIDACION,LLAMADA A TITULAR-PROMESA DE PAGO LIQUIDACION,2020-01-31,2250.0,6674234755 se agenda pago por:2250 al dia 318...,2020,1,5063290226839017-1-2020


In [25]:
#Concatenamos los pagos de brad
pagos_brad = pd.concat([pagos_brad_c0,pagos_brad_extra],ignore_index=True)
pagos_brad = pagos_brad.loc[pagos_brad['FECHA'].map(str)!='00:00:00']
pagos_brad['FECHA'] = pd.to_datetime(pagos_brad['FECHA'],dayfirst=False)
pagos_brad['YEAR'] = pagos_brad['FECHA'].dt.year
pagos_brad['MONTH'] = pagos_brad['FECHA'].dt.month
pagos_brad['credito_unico'] = pagos_brad['TARJETA'].map(str)+'-'+pagos_brad['MONTH'].map(str)+'-'+pagos_brad['YEAR'].map(str)
pagos_brad.head(1)

,PRODUCTO,TARJETA,PAGO,FECHA,PORCENTAJE,COMISION,BUCKET,DESPACHO,MES,YEAR,MONTH,credito_unico
0,PLCC,0002869002607825000,1542.0,2020-07-30,0.1,154.2,5,ENRIQUEZ,2020-07-01,2020,7,0002869002607825000-7-2020


In [26]:
pagos_brad2 = pd.pivot_table(pagos_brad,index=['TARJETA','FECHA','BUCKET','credito_unico','MES'],values=['PAGO','COMISION'],aggfunc=np.sum)
pagos_brad2 = pd.DataFrame(pagos_brad2.to_records())
pagos_brad2 = pagos_brad2.iloc[:,[0,6,1,3]]
pagos_brad2.columns = ['Cuenta','monto','Fecha pago','credito_unico']
pagos_brad2.head(1)

,Cuenta,monto,Fecha pago,credito_unico
0,0002869002605707000,1000.0,2020-10-16,0002869002605707000-10-2020


In [27]:
pagos_liv2 = pd.pivot_table(pagos_liv,index=['Cuenta','Fecha pago'],values=['monto'],aggfunc=np.sum)
pagos_liv2 = pd.DataFrame(pagos_liv2.to_records())
pagos_liv2['Fecha pago'] = pd.to_datetime(pagos_liv2['Fecha pago'],dayfirst=True)
pagos_liv2['YEAR'] = pagos_liv2['Fecha pago'].dt.year
pagos_liv2['MONTH'] = pagos_liv2['Fecha pago'].dt.month
pagos_liv2['credito_unico'] = pagos_liv2['Cuenta'].map(str)+'-'+pagos_liv2['MONTH'].map(str)+'-'+pagos_liv2['YEAR'].map(str)
pagos_liv2 = pagos_liv2.iloc[:,[0,2,1,5]]
pagos_liv2.colunms = ['Cuenta','monto','Fecha pago','credito_unico']
pagos_liv2.head(1)

,Cuenta,monto,Fecha pago,credito_unico
0,0000010000003513825,500.0,2020-07-24,0000010000003513825-7-2020


In [28]:
#Hacemos la union entre promesas y pagos
pagos_brad_liv = pd.concat([pagos_brad2,pagos_liv2],ignore_index=True)
promesas_pagos_brad = pd.merge(promesas_brad_liv,pagos_brad_liv,how='left',on='credito_unico')
promesas_pagos_brad = promesas_pagos_brad.drop_duplicates(subset=['credito','fecha_promesa','Fecha pago','monto','usuario'])
promesas_pagos_brad['usuario'] = [x.rstrip() for x in promesas_pagos_brad['usuario']]
promesas_pagos_brad.head(1)

,folio_gestion,firma_id,unegocio_id,credito,nombre_credito,telefono,tipo_telefono,fecha_gestion,usuario,nombre_usuario,...,accion_resultado,fecha_promesa,monto_promesa,comentarios,anio,mes,credito_unico,Cuenta,monto,Fecha pago
0,6604600,1,4,5063290226839017,ORONA PONCE ANGEL ALFREDO,6674234755,celular,2020-01-31 20:43:22,YHERNANDEZBA,ALEJANDRA YESENIA HERNANDEZ BITE,...,LLAMADA A TITULAR-PROMESA DE PAGO LIQUIDACION,2020-01-31,2250.0,6674234755 se agenda pago por:2250 al dia 318...,2020,1,5063290226839017-1-2020,5063290226839017,2250.0,2020-01-31


In [29]:
gestor_mes = pd.pivot_table(promesas_pagos_brad,index=['usuario'],values=['monto_promesa'],columns=['mes'],aggfunc=np.sum)
gestor_mes = pd.DataFrame(gestor_mes.to_records())
gestor_mes = gestor_mes[gestor_mes.isnull().sum(axis=1) < 8]

In [52]:
gestor_mes = gestor_mes[np.abs(gestor_mes - gestor_mes.mean()) <= (3 * gestor_mes.std())]
#cols = ['']
gestor_mes = gestor_mes.replace({'0':np.nan, 0:np.nan})

In [53]:
def cap_data(df):
    for col in df.columns:
        if (((df[col].dtype)=='float64') | ((df[col].dtype)=='int64')):
            percentiles = df[col].quantile([0.25,0.75]).values
            df[col][df[col] <= percentiles[0]] = percentiles[0]
            df[col][df[col] >= percentiles[1]] = percentiles[1]
        else:
            df[col]=df[col]
    return df

In [54]:
gestor_mes_final=cap_data(gestor_mes)

In [56]:
gestor_mes['Mean'] = gestor_mes.mean(numeric_only=True, axis=1).round(2)
gestor_mes.loc['mean'] = gestor_mes.mean().round(2)
gestor_mes

,usuario,"('monto_promesa', 1)","('monto_promesa', 2)","('monto_promesa', 3)","('monto_promesa', 4)","('monto_promesa', 5)","('monto_promesa', 6)","('monto_promesa', 7)","('monto_promesa', 8)","('monto_promesa', 9)","('monto_promesa', 10)","('monto_promesa', 11)","('monto_promesa', 12)",Mean
9,NaN,254214.7500,260698.00,137912.875,123049.8125,114182.25,183297.000,186386.00,248946.000,159722.000,138756.00,NaN,NaN,180716.47
17,NaN,256881.0000,153713.89,137912.875,123049.8125,114182.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,157147.97
21,NaN,NaN,NaN,NaN,NaN,NaN,138945.275,144034.50,173362.750,124911.625,138756.00,103777.2075,13611.00,119628.34
40,NaN,489486.5625,347023.00,334883.750,352018.9375,270761.25,382334.250,395721.75,173362.750,NaN,NaN,NaN,NaN,343199.03
42,NaN,302373.0000,232215.00,137912.875,239301.0000,114182.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,205196.82
44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200047.00,322264.000,239206.000,203941.00,161664.5000,15738.38,190476.81
47,NaN,489486.5625,170177.51,301623.000,271917.0000,NaN,138945.275,NaN,392065.000,225493.000,264684.00,183301.3225,5843.50,244353.62
51,NaN,NaN,NaN,NaN,123049.8125,117775.00,NaN,243103.41,392817.625,347877.000,222368.00,111066.0000,NaN,222579.55
52,NaN,383389.0000,371670.00,334883.750,318995.0000,270761.25,382334.250,395721.75,173362.750,NaN,NaN,NaN,NaN,328889.72
57,NaN,NaN,381505.00,162885.000,138760.0000,215728.00,169104.000,NaN,NaN,NaN,NaN,NaN,NaN,213596.40


# Performance promedio

In [34]:
#Hacemos la consulta referente al performance del Gestor
performance_all = consulta_sql('procesos_externos.all_rpt_agent_performance',20200101,mes)
performance_all.columns = ['fecha','usuario','nombre_usuario','user_group','llamadas','aht','ocupacion','adherencia','login_vici','logout_vici','staff_time','wait_time','talk_time','dispo_time','pause','login','manual','gestion','agua','alimentos','sanitario','retroalimentacion','tranferencia','validacion','whatsapp','incumplimiento','recordatorios','capacitacion','sistemas','lagged','tipificando','*']

In [35]:
def to_hour(df,cols):
    df[cols] = [datetime.strptime(x,'%H:%M:%S') for x in df[cols]]
    return df

In [36]:
performance_liv_brad = performance_all.loc[(performance_all['user_group']=='LIVERPOOL') | (performance_all['user_group']=='BRADESCARD')]
performance_liv_brad[['ocupacion','adherencia']] = percent_tonumeric(performance_liv_brad,['ocupacion','adherencia'])

In [37]:
performance_liv_brad['hora_login'] = [int(x[:2]) for x in performance_liv_brad['login_vici']]
performance_liv_brad['hora_logout'] = [int(x[:2]) for x in performance_liv_brad['logout_vici']]
performance_liv_brad['turno'] = np.where((performance_liv_brad['hora_logout']<=14)&(performance_liv_brad['hora_logout']-performance_liv_brad['hora_login']<=8),'M',
                                    np.where((performance_liv_brad['hora_logout']>17)&(performance_liv_brad['hora_logout']-performance_liv_brad['hora_login']<=8),'V','Mixto'))

In [44]:
performance_liv_brad['horas_trabajadas'] = performance_liv_brad['hora_logout']-performance_liv_brad['hora_login']
performance_liv_brad['llamadas_hora'] = performance_liv_brad['llamadas']/performance_liv_brad['horas_trabajadas']
performance_liv_brad['llamadas_hora'] = performance_liv_brad['llamadas_hora'].round()
performance_liv_brad.head(1)

,fecha,usuario,nombre_usuario,user_group,llamadas,aht,ocupacion,adherencia,login_vici,logout_vici,...,capacitacion,sistemas,lagged,tipificando,*,hora_login,hora_logout,turno,horas_trabajadas,llamadas_hora
1,2020-10-01,AGUILARMAVA,AGUILAR MACEDO VALERIA,BRADESCARD,253,00:00:30,12.84,24.43,08:03:49,20:58:28,...,00:00:00,00:00:00,00:00:00,00:10:09,00:00:00,8,20,Mixto,12,21.0


In [49]:
control_performance_brad_turno = pd.pivot_table(performance_liv_brad,index=['turno'],values=['llamadas','ocupacion','horas_trabajadas'],aggfunc=[np.sum,np.mean,'count',min,max])
control_performance_brad_turno = to_dataframe(control_performance_brad_turno)
#control_performance_brad_turno = control_performance_brad_turno.iloc[:,[0,1,2,3,4,5,6,7,8,11,12,13,14,15,16]]
#control_performance_brad_turno.columns = ['Turno','SumHorasTrabajadas','SumLlamadasDia','SumOcupacion','PromedioHorasTrabajadas','PromedioLlamadas','PromedioOcupacion','DiasAsitencia','MinHorasTrabajadas','MinLlamadas','MinOcupacion','MaxHorasTrabajadas','MaxLlamadas','MaxOcupacion']
control_performance_brad_turno

,turno,"('sum', 'horas_trabajadas')","('sum', 'llamadas')","('sum', 'ocupacion')","('mean', 'horas_trabajadas')","('mean', 'llamadas')","('mean', 'ocupacion')","('count', 'horas_trabajadas')","('count', 'llamadas')","('count', 'ocupacion')","('min', 'horas_trabajadas')","('min', 'llamadas')","('min', 'ocupacion')","('max', 'horas_trabajadas')","('max', 'llamadas')","('max', 'ocupacion')"
0,M,7350,219063,42723.82,6.369151,189.829289,37.022374,1154,1154,1154,0,0,0.00,8,591,81.00
1,Mixto,7428,208384,24036.67,11.169925,313.359398,36.145368,665,665,665,0,0,0.00,14,1236,75.24
2,V,3379,101517,21567.50,6.375472,191.541509,40.693396,530,530,530,1,12,7.17,8,487,65.90


In [50]:
control_performance_brad_turno2 = control_performance_brad_turno[np.abs(control_performance_brad_turno - control_performance_brad_turno.mean()) <= (3 * control_performance_brad_turno.std())]
#cols = ['']
control_performance_brad_turno2 = control_performance_brad_turno2.replace({'0':np.nan, 0:np.nan})

In [51]:
control_performance_brad_turno.loc['mean'] = control_performance_brad_turno.mean().round(2)
control_performance_brad_turno

,turno,"('sum', 'horas_trabajadas')","('sum', 'llamadas')","('sum', 'ocupacion')","('mean', 'horas_trabajadas')","('mean', 'llamadas')","('mean', 'ocupacion')","('count', 'horas_trabajadas')","('count', 'llamadas')","('count', 'ocupacion')","('min', 'horas_trabajadas')","('min', 'llamadas')","('min', 'ocupacion')","('max', 'horas_trabajadas')","('max', 'llamadas')","('max', 'ocupacion')"
0,M,7350.00,219063.00,42723.82,6.369151,189.829289,37.022374,1154.0,1154.0,1154.0,0.00,0.0,0.00,8.0,591.00,81.00
1,Mixto,7428.00,208384.00,24036.67,11.169925,313.359398,36.145368,665.0,665.0,665.0,0.00,0.0,0.00,14.0,1236.00,75.24
2,V,3379.00,101517.00,21567.50,6.375472,191.541509,40.693396,530.0,530.0,530.0,1.00,12.0,7.17,8.0,487.00,65.90
mean,NaN,6052.33,176321.33,29442.66,7.970000,231.580000,37.950000,783.0,783.0,783.0,0.33,4.0,2.39,10.0,771.33,74.05


In [57]:
writer = pd.ExcelWriter(''+ruta_guardar+'/Gestores/BradLiv/Puntos BradLiv '+now+'.xlsx',engine='xlsxwriter')
gestor_mes.to_excel(writer,'BSC',index=False,header=True)
control_performance_brad_turno.to_excel(writer,'PerformanceDetallado',index=False,header=True)
control_performance_brad_turno2.to_excel(writer,'PerformanceTurno',index=False,header=True)

writer.save()